In [8]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt

In [3]:
current_tiles_file = '/global/cfs/cdirs/desi/users/desiproc/surveyops/trunk/ops/tiles-main.ecsv'

In [4]:
current_tiles = Table.read(current_tiles_file)

In [5]:
current_tiles

TILEID,PASS,RA,DEC,PROGRAM,IN_DESI,PRIORITY,STATUS,EBV_MED,DESIGNHA,DONEFRAC,AVAILABLE,PRIORITY_BOOSTFAC
int64,int32,float64,float64,str6,bool,float64,str8,float32,float32,float32,bool,float64
1000,0,238.282,0.0,DARK,True,1.458e+00,done,0.101,15.91,1.1687,True,1.000
1001,0,270.0,58.282,DARK,True,9.521e-01,done,0.044,31.26,1.0143,True,1.000
1002,0,0.0,31.717,DARK,True,1.153e+00,done,0.049,16.62,1.2018,True,1.000
1003,0,180.0,31.717,DARK,True,8.483e-01,unobs,0.020,-10.34,0.0000,False,1.000
1004,0,180.0,69.094,DARK,True,6.503e-01,unobs,0.015,-8.02,0.0000,True,1.000
1005,0,200.905,0.0,DARK,True,1.458e+00,done,0.026,2.82,0.8615,True,1.000
1006,0,20.905,0.0,DARK,True,1.458e+00,done,0.033,12.00,0.9638,True,1.000
1007,0,225.0,35.264,DARK,True,1.168e+00,done,0.014,11.94,1.1197,True,1.000
1008,0,159.094,0.0,DARK,True,1.458e+00,done,0.062,-10.78,0.9958,True,1.000


In [51]:
in_desi = current_tiles['IN_DESI']
is_dark = current_tiles['PROGRAM']=='DARK'
is_bright = current_tiles['PROGRAM']=='BRIGHT'
is_done = current_tiles['STATUS']=='done'
is_available = current_tiles['AVAILABLE']

next_bright_tiles = current_tiles[in_desi & is_bright & ~is_done]
next_dark_tiles = current_tiles[in_desi & is_dark & ~is_done]

print(len(next_bright_tiles), len(next_dark_tiles))
program_probe = {'DARK':'ELG', 'BRIGHT':'BGS'}

1961 3368


In [60]:
f = open('run_next_dark_tiles.sh', 'w')
for line in next_dark_tiles:
    exec_line = f"fba_launch --doclean n --dr dr9 --dtver 1.1.1 --forcetileid y --gaiadr gaiadr2 --goaltime 1000.0 --goaltype {line['PROGRAM']}" 
    exec_line += f" --ha {line['DESIGNHA']} --hdr_faprgrm {line['PROGRAM']} --hdr_survey main --lookup_sky_source ls "
    exec_line += f" --margin_gfa 0.4 --margin_petal 0.4 --margin_pos 0.05 --mintfrac 0.85 --mtltime 2024-04-15T00:00:00+00:00 " 
    exec_line += f" --outdir /global/cfs/cdirs/desi/users/forero/fba_sim_next_tiles_20240415 --pmcorr n --pmtime_utc_str 2024-04-15T00:00:00+00:00 "
    exec_line += f" --program {line['PROGRAM']} --rundate 2024-04-15T00:00:00+00:00 --sbprof {program_probe[line['PROGRAM']]} "
    exec_line += f" --sky_per_petal 40 --sky_per_slitblock 1 --standards_per_petal 10 --steps tiles,sky,gfa,targ,scnd,too,fa,zip,move --survey main" 
    exec_line += f" --tiledec {line['DEC']} --tileid {line['TILEID']} --tilera {line['RA']} --worldreadable\n"
    f.write(exec_line)
f.close()


f = open('run_next_bright_tiles.sh', 'w')
for line in next_bright_tiles:
    exec_line = f"fba_launch --doclean n --dr dr9 --dtver 1.1.1 --forcetileid y --gaiadr gaiadr2 --goaltime 1000.0 --goaltype {line['PROGRAM']}" 
    exec_line += f" --ha {line['DESIGNHA']} --hdr_faprgrm {line['PROGRAM']} --hdr_survey main --lookup_sky_source ls "
    exec_line += f" --margin_gfa 0.4 --margin_petal 0.4 --margin_pos 0.05 --mintfrac 0.85 --mtltime 2024-04-15T00:00:00+00:00 " 
    exec_line += f" --outdir /global/cfs/cdirs/desi/users/forero/fba_sim_next_tiles_20240415 --pmcorr n --pmtime_utc_str 2024-04-15T00:00:00+00:00 "
    exec_line += f" --program {line['PROGRAM']} --rundate 2024-04-15T00:00:00+00:00 --sbprof {program_probe[line['PROGRAM']]} "
    exec_line += f" --sky_per_petal 40 --sky_per_slitblock 1 --standards_per_petal 10 --steps tiles,sky,gfa,targ,scnd,too,fa,zip,move --survey main" 
    exec_line += f" --tiledec {line['DEC']} --tileid {line['TILEID']} --tilera {line['RA']} --worldreadable\n"
    f.write(exec_line)
f.close()

In [62]:
(1961+3368)*10/3600.0

14.802777777777777